In [2]:
# Milwaukee Restaurant Reddit Data Collection

import requests
import requests.auth
import praw
import pandas as pd
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from datetime import datetime, timedelta
import re
from collections import Counter


In [ ]:
#pull up to 8 google reviews for each restaurant (60 total)
df=pd.read_csv('mke_reviews.csv')
df=df[['title','neighborhood','description','phone','reviews/0/text','reviews/0/publishedAtDate','reviews/1/text','reviews/1/publishedAtDate',
       'reviews/2/text','reviews/2/publishedAtDate',
       'reviews/3/text','reviews/3/publishedAtDate','reviews/4/text',
       'reviews/4/publishedAtDate','reviews/5/text','reviews/5/publishedAtDate','reviews/6/text','reviews/6/publishedAtDate',
       'reviews/7/text','reviews/7/publishedAtDate']]
rest_list=df['title'].to_list()
col_names=['title','neighborhood','description','phone','reviews/0/text','reviews/0/publishedAtDate','reviews/1/text','reviews/1/publishedAtDate',
       'reviews/2/text','reviews/2/publishedAtDate',
       'reviews/3/text','reviews/3/publishedAtDate','reviews/4/text',
       'reviews/4/publishedAtDate','reviews/5/text','reviews/5/publishedAtDate','reviews/6/text','reviews/6/publishedAtDate',
       'reviews/7/text','reviews/7/publishedAtDate']

In [ ]:

reddit = praw.Reddit(
    client_id="KvrTjF5kzjUZAu_RrKlvzw",
    client_secret="VqtjIjPhtfbZQVrbDm7KJPfl1BVPnw",
    user_agent="milwaukee_food_tracker",
)

print(f"Read only mode: {reddit.read_only}")


# r/milwaukee - main Milwaukee subreddit
# r/wisconsin - broader Wisconsin discussions

RESTAURANTS = rest_list

FOOD_KEYWORDS = [
    "restaurant", "food", "brunch", "dinner", "lunch",
    "bar","drinks", "brewery", "cafe", "pizza", "burger", "taco","sandwich","vegetarian"
]


Read only mode: True


In [ ]:

def find_restaurants_in_text(text, restaurants):
    text_lower = text.lower()
    matches = [r for r in restaurants if r.lower() in text_lower]
    return matches


In [ ]:
#grab  posts from subreddit, KEEP ONLY posts that actually mention
    at least one restaurant from RESTAURANTS.
def collect_reddit_posts(subreddit_name, time_filter="month", limit=500):

    subreddit = reddit.subreddit(subreddit_name)

    data = []

    SEARCH_TERMS = RESTAURANTS + FOOD_KEYWORDS

    seen_ids = set()

    for term in SEARCH_TERMS:
        print(f"> Searching '{term}' in r/{subreddit_name}")
        try:
            submissions = subreddit.search(term, time_filter=time_filter, limit=limit)

            for submission in submissions:
                if submission.id in seen_ids:
                    continue
                
                full_text = (submission.title or "") + " " + (submission.selftext or "")
                found = find_restaurants_in_text(full_text, RESTAURANTS)

                if len(found) == 0:
                    continue 

                seen_ids.add(submission.id)

                data.append({
                    "ID": submission.id,
                    "Title": submission.title,
                    "Body": submission.selftext,
                    "Restaurants_Mentioned": list(set(found)),
                    "Score": submission.score,
                    "Num_Comments": submission.num_comments,
                    "Created_UTC": datetime.fromtimestamp(submission.created_utc),
                    "URL": submission.url,
                    "Subreddit": subreddit_name,
                })

            time.sleep(1)

        except Exception as e:
            print(f"Error searching {term}: {e}")

    return data


In [ ]:
def collect_restaurant_comments(submission_ids, restaurants):#only collects comments mentioning at least 1 restaurant

    comments_data = []

    for sub_id in submission_ids:
        try:
            submission = reddit.submission(id=sub_id)
            submission.comments.replace_more(limit=0)

            for comment in submission.comments.list():
                if not comment.body:
                    continue

                found = find_restaurants_in_text(comment.body, restaurants)
                if len(found) == 0:
                    continue  # ❌ skip comments with no restaurant mention

                comments_data.append({
                    "Submission_ID": sub_id,
                    "Comment_Body": comment.body,
                    "Restaurants_Mentioned": list(set(found)),
                    "Comment_Score": comment.score,
                    "Created_UTC": datetime.fromtimestamp(comment.created_utc),
                })

            time.sleep(0.5)

        except Exception as e:
            print(f"error, for {sub_id}: {e}")

    return comments_data


In [ ]:
all_data = []

all_data.extend(collect_reddit_posts("milwaukee", time_filter="year", limit=200))

all_data.extend(collect_reddit_posts("wisconsin", time_filter="year", limit=200))

df_posts = pd.DataFrame(all_data)

len(df_posts)



--- Collecting from r/milwaukee ---
> Searching 'East Town Kitchen + Bar' in r/milwaukee
> Searching 'The National Cafe' in r/milwaukee
> Searching 'Onesto' in r/milwaukee
> Searching 'Easy Tyger | An Asian Gastropub' in r/milwaukee
> Searching 'La Caribeña The best Caribbean family restaurant' in r/milwaukee
> Searching 'State Street Pizza Pub' in r/milwaukee
> Searching 'Screaming Tuna Milwaukee' in r/milwaukee
> Searching 'Elsa's On the Park' in r/milwaukee
> Searching 'Lucky Ginger' in r/milwaukee
> Searching 'La Dama | Mexican Kitchen & Bar' in r/milwaukee
> Searching 'Michael’s Family Restaurant' in r/milwaukee
> Searching 'Botanas Restaurant' in r/milwaukee
> Searching 'Blue Bat Kitchen & Tequilaria' in r/milwaukee
> Searching 'Orenda Cafe' in r/milwaukee
> Searching 'Copper on King' in r/milwaukee
> Searching 'Blue Star Cafe' in r/milwaukee
> Searching 'Harbor House' in r/milwaukee
> Searching 'The Knick' in r/milwaukee
> Searching 'The Diplomat' in r/milwaukee
> Searching 'Fi

In [14]:
df_posts.to_csv("milwaukee_restaurants_posts.csv")

In [16]:
post_ids = df_posts["ID"].tolist()

df_comments = pd.DataFrame(
    collect_restaurant_comments(post_ids, RESTAURANTS)
)

df_comments.to_csv("milwaukee_restaurants_comments.csv", index=False)

print(f"Collected {len(df_comments)} restaurant-specific comments.")


Collected 106 restaurant-specific comments.


In [ ]:
#google reviews flattened -> long

cols = [
    "title", "neighborhood", "description", "phone",
    *[f"reviews/{i}/text" for i in range(8)],
    *[f"reviews/{i}/publishedAtDate" for i in range(8)]
]
dft = df[cols]

def reshape_google_reviews(dft):
    rows = []

    for _, row in dft.iterrows():
        restaurant = row["title"]

        for i in range(8):
            text_col = f"reviews/{i}/text"
            date_col = f"reviews/{i}/publishedAtDate"

            text = row.get(text_col)
            date = row.get(date_col)

            # skip missing reviews
            if pd.isna(text):
                continue

            rows.append({
                "restaurant": restaurant,
                "text": text,
                "date": date,
                "source": "google"
            })

    return pd.DataFrame(rows)

df_reviews = reshape_google_reviews(dft)
df_reviews.head()

,restaurant,text,date,source
0,East Town Kitchen + Bar,I was really worried this was going to be a ba...,2025-11-23T21:28:07.596Z,google
1,East Town Kitchen + Bar,Food was terrible. Plates were never cleared. ...,2025-11-23T15:33:30.799Z,google
2,East Town Kitchen + Bar,Great beer from Taylor!,2025-11-22T20:23:23.320Z,google
3,East Town Kitchen + Bar,Taylor did a great job accommodating our picky...,2025-11-22T19:35:39.134Z,google
4,East Town Kitchen + Bar,Solid selection of beers on tap!,2025-11-22T19:34:58.316Z,google


In [ ]:

POSITIVE_WORDS = {
    'good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic', 'delicious',
    'tasty', 'yummy', 'fresh', 'perfect', 'love', 'loved', 'best', 'awesome',
    'favorite', 'recommend', 'must', 'outstanding', 'superb', 'incredible',
    'exceptional', 'phenomenal', 'mouthwatering', 'scrumptious', 'delectable',
    'savory', 'flavorful', 'tender', 'juicy', 'crispy', 'creamy', 'satisfying',
    'hearty', 'authentic', 'cozy', 'friendly', 'attentive', 'prompt', 'clean',
    'affordable', 'reasonable', 'worth', 'stellar', 'top-notch', 'divine',
    'perfectly', 'well', 'exceeded', 'surpassed', 'impressed'
}

NEGATIVE_WORDS = {
    'bad', 'terrible', 'awful', 'horrible', 'disappointing', 'poor', 'worst',
    'overpriced', 'expensive', 'rude', 'slow', 'dirty', 'cold', 'bland',
    'tasteless', 'dry', 'burnt', 'undercooked', 'salty', 'soggy', 'greasy',
    'mediocre', 'average', 'meh', 'okay', 'disgusting', 'inedible',
    'unprofessional', 'unfriendly', 'ignored', 'wait', 'crowded', 'noisy',
    'smelly', 'stale', 'fake', 'disappointed', 'avoid', 'waste', 'rip-off',
    'never', 'again', 'sorry', 'unfortunately', 'mess', 'chaotic', 'understaffed'
}

INTENSIFIERS = {
    'very', 'really', 'extremely', 'incredibly', 'absolutely', 'totally',
    'completely', 'utterly', 'especially', 'particularly', 'exceptionally'
}

NEGATION_WORDS = {
    'not', "isn't", "aren't", "wasn't", "weren't", "don't", "doesn't", "didn't",
    "won't", "wouldn't", "couldn't", "shouldn't", "can't", "cannot",
    'never', 'no', 'none', 'nothing', 'nobody', 'nowhere', 'neither', 'nor'
}

def simple_sentiment_analysis(text):
    """
    Simple lexicon-based sentiment analysis with negation handling
    Returns sentiment score between -1 (very negative) and 1 (very positive)
    """
    if not isinstance(text, str) or not text.strip():
        return 0
    
    text_lower = text.lower()
    words = re.findall(r'\b\w+\b', text_lower)
    
    if not words:
        return 0
    
    sentiment_score = 0
    negate_next = False
    intensify_next = False
    
    for i, word in enumerate(words):
        # Check for negation
        if word in NEGATION_WORDS:
            negate_next = True
            continue
        
        # Check for intensifiers
        if word in INTENSIFIERS:
            intensify_next = True
            continue
        
        # Check for positive words
        if word in POSITIVE_WORDS:
            score = 1
            if intensify_next:
                score *= 1.5
            if negate_next:
                score *= -1
            sentiment_score += score
        
        # Check for negative words
        elif word in NEGATIVE_WORDS:
            score = -1
            if intensify_next:
                score *= 1.5
            if negate_next:
                score *= -1
            sentiment_score += score
        
        # Reset flags after each word
        negate_next = False
        intensify_next = False
    
    # Normalize the score to be between -1 and 1
    normalized_score = sentiment_score / len(words)
    return max(-1, min(1, normalized_score))

def categorize_sentiment(score):
    """Categorize sentiment score into label"""
    if score > 0.1:
        return "positive"
    elif score < -0.1:
        return "negative"
    else:
        return "neutral"


In [21]:
df_posts['sentiment_score'] = df_posts['Body'].apply(simple_sentiment_analysis)
df_posts['sentiment'] = df_posts['sentiment_score'].apply(categorize_sentiment)

#sentiment analysis comments
df_comments['sentiment_score'] = df_comments['Comment_Body'].apply(simple_sentiment_analysis)
df_comments['sentiment'] = df_comments['sentiment_score'].apply(categorize_sentiment)

#  sentiment analysis google reviews
df_reviews['sentiment_score'] = df_reviews['text'].apply(simple_sentiment_analysis)
df_reviews['sentiment'] = df_reviews['sentiment_score'].apply(categorize_sentiment)

In [22]:
df_comments = df_comments[df_comments["Restaurants_Mentioned"].str.len() == 1].assign(
    Restaurants_Mentioned=lambda x: x["Restaurants_Mentioned"].str[0]
)

df_posts = df_posts[df_posts["Restaurants_Mentioned"].str.len() == 1].assign(
    Restaurants_Mentioned=lambda x: x["Restaurants_Mentioned"].str[0]
)


In [29]:
df_reviews.to_csv('google_reviews_long_format.csv')
df_posts.to_csv('milwaukee_restaurants_posts.csv')
df_comments.to_csv('milwaukee_restaurants_comments.csv')